In [51]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

# https://www.flashscore.com.br/jogo/dhuWUY6g/#/resumo-de-jogo/estatisticas-de-jogo/0

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install())
)  # Configura o driver do Chrome
driver.get("https://www.flashscore.com.br/")  # Acessa o site
print(driver.title)  # Imprime o título da página

WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable((By.XPATH, """//*[@id="onetrust-accept-btn-handler"]"""))
).click()

Resultados de futebol ao vivo, Brasileirão, Série B, Série C | Flashscore.com.br


In [7]:
driver.title

'Resultados de futebol ao vivo, Brasileirão, Série B, Série C | Flashscore.com.br'

In [4]:
url_match = (
    "https://www.flashscore.com.br/jogo/dhuWUY6g/#/resumo-de-jogo/resumo-de-jogo"
)
driver.get(url_match)

In [5]:
driver.title

'SPO 2-1 CEA | Sport - Ceará | Sumário'

In [6]:
current_url = driver.current_url
print("URL atual:", current_url)
match_stats = (
    driver.current_url.split("#")[0] + "#/resumo-de-jogo/estatisticas-de-jogo/"
)
print("URL direta para estatisticas:", match_stats)
# driver.get(match_stats) # Acessa as estatisticas

URL atual: https://www.flashscore.com.br/jogo/dhuWUY6g/#/resumo-de-jogo/resumo-de-jogo
URL direta para estatisticas: https://www.flashscore.com.br/jogo/dhuWUY6g/#/resumo-de-jogo/estatisticas-de-jogo/


In [8]:
tabs_div = driver.find_element(By.CLASS_NAME, "filterOver")
# captura toda a div com a class filterOver


anchors = tabs_div.find_elements(By.TAG_NAME, "a")
# captura todos os links da div
print("Quantidade de links encontrados:", len(anchors))

anchors[1].screenshot("debug.png")
# Conferindo se o elemento certo foi capturado


anchors[1].click()  # Acessa as estatisticas

Quantidade de links encontrados: 3


In [9]:
match_detais = {}
soup = BeautifulSoup(driver.page_source, "html.parser")

In [10]:
print(soup.find("span", class_="tournamentHeader__country").text)
# BRASIL: Série B - Rodada 30

match_header = soup.find("span", class_="tournamentHeader__country").text
match_detais["country"] = match_header.split(":")[0]
match_detais["tournament_name"] = match_header.split(": ")[1].split(" - ")[0]
match_detais["tournament_round"] = match_header.split("Rodada ")[1]

print(match_detais)

BRASIL: Série B - Rodada 30
{'country': 'BRASIL', 'tournament_name': 'Série B', 'tournament_round': '30'}


In [11]:
print(soup.find("div", class_="duelParticipant__startTime").text)

match_datetime = soup.find("div", class_="duelParticipant__startTime").text
match_detais["date"], match_detais["time"] = match_datetime.split(" ")

print(match_detais)

07.10.2024 21:00
{'country': 'BRASIL', 'tournament_name': 'Série B', 'tournament_round': '30', 'date': '07.10.2024', 'time': '21:00'}


In [12]:
match_detais["home_team"] = soup.find("div", class_="duelParticipant__home").text
match_detais["away_team"] = soup.find("div", class_="duelParticipant__away").text

match_score = soup.find("div", class_="detailScore__wrapper").text
match_detais["home_score"], match_detais["away_score"] = match_score.split("-")

print(match_detais)

{'country': 'BRASIL', 'tournament_name': 'Série B', 'tournament_round': '30', 'date': '07.10.2024', 'time': '21:00', 'home_team': 'Sport', 'away_team': 'Ceará', 'home_score': '2', 'away_score': '1'}


In [13]:
all_stats = soup.find("div", class_="section").find_all("div", recursive=False)
all_stats

[<div class="wcl-row_OFViZ" data-testid="wcl-statistics"><div class="wcl-category_ITphf"><div class="wcl-value_IuyQw wcl-homeValue_-iJBW" data-testid="wcl-statistics-value"><strong class="wcl-simpleText_Asp-0 wcl-webTypeSimpleText01_LX1Fm wcl-bold_roH-0" data-testid="wcl-scores-simpleText1">51%</strong></div><div class="wcl-category_7qsgP" data-testid="wcl-statistics-category"><strong class="wcl-simpleText_Asp-0 wcl-webTypeSimpleText01_LX1Fm wcl-bold_roH-0" data-testid="wcl-scores-simpleText1">Posse de Bola</strong></div><div class="wcl-value_IuyQw wcl-awayValue_rQvxs" data-testid="wcl-statistics-value"><strong class="wcl-simpleText_Asp-0 wcl-webTypeSimpleText01_LX1Fm wcl-bold_roH-0" data-testid="wcl-scores-simpleText1">49%</strong></div></div><div class="wcl-charts_mNAIV"><div class="wcl-chart_qyFXG wcl-home_NAPaA"><div class="wcl-betterSideBackground_mhWVV" data-testid="wcl-statistics-chart-home" style="width: 51%;"></div></div><div class="wcl-chart_qyFXG"><div class="wcl-worseSideOr

In [50]:
first_stats_divs = all_stats[0].find("div").find_all("div", recursive=False)
for div in first_stats_divs:
    print(div.text)

51%
Posse de Bola
49%


In [16]:
for stats in all_stats:
    stats_divs = [
        div.text for div in stats.find("div").find_all("div", recursive=False)
    ]
    category = stats_divs[1].lower().replace(" ", "_")
    match_detais[category] = [stats_divs[0], stats_divs[2]]

match_detais

{'country': 'BRASIL',
 'tournament_name': 'Série B',
 'tournament_round': '30',
 'date': '07.10.2024',
 'time': '21:00',
 'home_team': 'Sport',
 'away_team': 'Ceará',
 'home_score': '2',
 'away_score': '1',
 'posse_de_bola': ['51%', '49%'],
 'tentativas_de_gol': ['14', '13'],
 'chutes_no_gol': ['3', '2'],
 'chutes_para_fora': ['8', '9'],
 'chutes_bloqueados': ['3', '2'],
 'faltas_cobradas': ['9', '9'],
 'escanteios': ['4', '6'],
 'impedimentos': ['2', '5'],
 'laterais_cobrados': ['27', '27'],
 'defesas_do_goleiro': ['1', '1'],
 'faltas': ['9', '9'],
 'cartões_amarelos': ['6', '5'],
 'passes_totais': ['426', '361'],
 'passes_completados': ['354', '262']}

## Extraindo todo um campeonato


In [39]:
def load_matchs_list(driver):
    while True:
        try:
            btn_load_more = WebDriverWait(driver, 4).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "event__more"))
            )

            # Centraliza o elemento na página.
            driver.execute_script(
                "arguments[0].scrollIntoView({block: 'center'});", btn_load_more
            )

            time.sleep(2)
            btn_load_more.click()
            time.sleep(2)
        except:
            print("Sem mais partidas para carregar!")
            return 1

In [40]:
def get_matchs_links(driver):
    match_links = [
        div.get_attribute("href") + "/estatisticas-de-jogo/"
        for div in driver.find_elements(By.CLASS_NAME, "eventRowLink")
    ]
    return match_links

In [49]:
def load_and_extract_match_data(match_link, driver):
    match_data = {}
    match_data["match_link"] = match_link
    try:
        driver.get(match_link)

        WebDriverWait(driver, 4).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "section"))
        )
        print("Processando:", driver.title)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        match_header = soup.find("span", class_="tournamentHeader__country").text
        match_data["country"] = match_header.split(":")[0]
        match_data["tournament_name"] = match_header.split(": ")[1].split(" - ")[0]
        match_data["tournament_round"] = match_header.split("Rodada ")[1]

        match_datetime = soup.find("div", class_="duelParticipant__startTime").text
        match_data["date"], match_data["time"] = match_datetime.split(" ")

        match_data["home_team"] = soup.find("div", class_="duelParticipant__home").text
        match_data["away_team"] = soup.find("div", class_="duelParticipant__away").text

        match_score = soup.find("div", class_="detailScore__wrapper").text
        match_data["home_score"], match_data["away_score"] = match_score.split("-")

        all_stats = soup.find("div", class_="section").find_all("div", recursive=False)
        for stats in all_stats:
            stats_divs = [
                div.text for div in stats.find("div").find_all("div", recursive=False)
            ]
            category = stats_divs[1].lower().replace(" ", "_")
            match_data[category] = [stats_divs[0], stats_divs[2]]

    except Exception as e:
        print("Erro em:", match_link)
        print(str(e))

    return match_data

In [52]:
def collect_league_statistics(league_url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(league_url)
    load_matchs_list(driver)
    match_links = get_matchs_links(driver)

    matchs_data = []
    for match_link in match_links:
        matchs_data.append(load_and_extract_match_data(match_link, driver))

    df = pd.DataFrame(matchs_data)
    df.to_csv("matchs.csv", index=False)
    return df

league_url = (
    "https://www.flashscore.com.br/futebol/brasil/brasileirao-betano-2023/resultados/"
)
collect_league_statistics(league_url)

Sem mais partidas para carregar!
Processando: BAH 4-1 ATL | Bahia - Atlético-MG | Estatísticas
Processando: CFC 0-2 COR | Coritiba - Corinthians | Estatísticas
Processando: CRU 1-1 PAL | Cruzeiro - Palmeiras | Estatísticas
Processando: CUI 3-0 ATH | Cuiabá - Athletico-PR | Estatísticas
Processando: FLU 2-3 GRE | Fluminense - Grêmio | Estatísticas
Processando: INT 3-1 BOT | Internacional - Botafogo | Estatísticas
Processando: SAN 1-2 FOR | Santos - Fortaleza | Estatísticas
Processando: SAO 1-0 FLA | São Paulo - Flamengo | Estatísticas
Processando: VAS 2-1 BRA | Vasco - Red Bull Bragantino | Estatísticas
Processando: GOI 1-0 AMG | Goiás - América-MG | Estatísticas
Processando: AMG 3-2 BAH | América-MG - Bahia | Estatísticas
Processando: ATH 3-0 SAN | Athletico-PR - Santos | Estatísticas
Processando: BOT 0-0 CRU | Botafogo - Cruzeiro | Estatísticas
Processando: FOR 1-0 GOI | Fortaleza - Goiás | Estatísticas
Processando: GRE 1-0 VAS | Grêmio - Vasco | Estatísticas
Processando: BRA 1-0 CFC 

,match_link,country,tournament_name,tournament_round,date,time,home_team,away_team,home_score,away_score,...,laterais_cobrados,defesas_do_goleiro,faltas,cartões_amarelos,passes_totais,passes_completados,ataques,ataques_perigosos,cartões_vermelhos,desarmes
0,https://www.flashscore.com.br/jogo/zsLkfjIP/#/...,BRASIL,Série A,38,06.12.2023,21:30,Bahia,Atlético-MG,4,1,...,"[18, 14]","[4, 3]","[16, 10]","[2, 2]","[297, 534]","[235, 466]","[81, 111]","[45, 26]",NaN,NaN
1,https://www.flashscore.com.br/jogo/O6HoeW2J/#/...,BRASIL,Série A,38,06.12.2023,21:30,Coritiba,Corinthians,0,2,...,"[12, 14]","[4, 4]","[17, 12]","[1, 0]","[522, 680]","[456, 617]","[93, 104]","[63, 45]",NaN,NaN
2,https://www.flashscore.com.br/jogo/trh1mKqD/#/...,BRASIL,Série A,38,06.12.2023,21:30,Cruzeiro,Palmeiras,1,1,...,"[23, 21]","[4, 5]","[9, 19]","[1, 1]","[424, 381]","[356, 298]","[116, 60]","[38, 26]",NaN,NaN
3,https://www.flashscore.com.br/jogo/hM7HkAQt/#/...,BRASIL,Série A,38,06.12.2023,21:30,Cuiabá,Athletico-PR,3,0,...,"[10, 13]","[3, 3]","[10, 14]","[4, 3]","[402, 506]","[343, 436]","[76, 102]","[28, 62]","[1, 0]",NaN
4,https://www.flashscore.com.br/jogo/rTethrVs/#/...,BRASIL,Série A,38,06.12.2023,21:30,Fluminense,Grêmio,2,3,...,"[15, 15]","[2, 5]","[11, 11]","[3, 3]","[653, 295]","[599, 238]","[146, 68]","[76, 27]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,https://www.flashscore.com.br/jogo/vFRENgWQ/#/...,BRASIL,Série A,1,15.04.2023,18:30,Botafogo,São Paulo,2,1,...,"[19, 10]","[7, 1]","[20, 13]","[3, 2]","[293, 646]","[229, 574]","[85, 122]","[57, 41]",NaN,"[10, 16]"
376,https://www.flashscore.com.br/jogo/trlpwZVD/#/...,BRASIL,Série A,1,15.04.2023,18:30,Fortaleza,Internacional,1,1,...,"[24, 17]","[5, 5]","[18, 13]","[2, 1]","[398, 474]","[319, 379]","[126, 110]","[73, 30]",NaN,"[17, 19]"
377,https://www.flashscore.com.br/jogo/hM7kHi9f/#/...,BRASIL,Série A,1,15.04.2023,18:30,Red Bull Bragantino,Bahia,2,1,...,"[26, 36]","[2, 5]","[20, 13]","[3, 1]","[436, 343]","[339, 248]","[127, 87]","[54, 39]",NaN,"[26, 11]"
378,https://www.flashscore.com.br/jogo/I7BDL60t/#/...,BRASIL,Série A,1,15.04.2023,16:00,América-MG,Fluminense,0,3,...,"[15, 13]","[5, 2]","[25, 9]","[3, 2]","[336, 442]","[267, 364]","[114, 56]","[95, 22]",NaN,"[18, 14]"
